# Flank Quickstart on Google Colab
## Run through this notebook to quickly set up an API endpoint and expose it on Flank

## 1. Spin up an API locally




In [7]:
%pip install -q fastapi uvicorn

In [8]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query

In [10]:
app = FastAPI()

# Dummy sales data
sales_data = [
    {"id": 1, "date": "2023-01-01", "amount": 100.0},
    {"id": 2, "date": "2023-01-02", "amount": 150.0},
    {"id": 3, "date": "2023-01-03", "amount": 200.0},
    {"id": 4, "date": "2023-01-04", "amount": 120.0},
    {"id": 5, "date": "2023-01-05", "amount": 180.0},
]

# Endpoint to generate sales report based on sale_id
@app.get("/api/reports/sales")
async def generate_sales_report(
    sale_id: int = Query(..., description="Sale ID to filter the sales report")
) -> dict:
    for sale in sales_data:
        if sale["id"] == sale_id:
            return sale
    return {"error": "Sale not found"}

### What happened here??

**fastapi** is an API framework. **uvicorn** is a server that listens for API requests. We just created a simple "endpoint" around the `sales_data` that exposes that code over HTTP.

You can test it out by going this link: http://localhost:8010/api/results

## 2. Expose your API on the public internet
**ngrok**: In a new tab, [log in or create a free ngrok account](https://dashboard.ngrok.com/tunnels/authtokens). Once you've logged in, navigate to the [authtoken page](https://dashboard.ngrok.com/get-started/your-authtoken) if you aren't already there. Copy the Authtoken at the top of the screen, and then navigate back here to the notebook.

Next, you'll be prompted to paste in that authtoken here.

In [9]:
print("Paste in your ngrok authtoken that you copied in the step above, then hit enter:")
conf.get_default().auth_token = getpass.getpass()

Paste in your ngrok authtoken that you copied in the step above, then hit enter:
··········


## 3. Spin up an API locally. Woo!

Next, set up your FastAPI endpoints. You can set up as many as you'd like, to do whatever you'd like, but here we'll set up an endpoint to generate a simple sales report.



## 4. Expose that API over the public internet. Hooray!
Now we'll expose your endpoint to the web. You can choose any port on your localhost that's open. Here we'll run and expose the API on port 8010.
We'll use `ngrok` to expose it, and `nest_asyncio` + `uvicorn` to run it.

In [15]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8010)"))

https://7nadloe4is4-496ff2e9c6d22116-8010-colab.googleusercontent.com/


In [ ]:
ngrok_tunnel = ngrok.connect(8010)
ngrok_url = ngrok_tunnel.public_url
print('See it working:', ngrok_url + "/api/reports/sales?sale_id=1")
print('Schema to upload to Flank:', ngrok_url + "/openapi.json")
nest_asyncio.apply()
uvicorn.run(app, port=8010)

See it working: https://c0ae-35-236-235-157.ngrok-free.app/api/reports/sales?sale_id=1
Schema to upload to Flank: https://c0ae-35-236-235-157.ngrok-free.app/openapi.json


INFO:     Started server process [926]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8010 (Press CTRL+C to quit)


INFO:     127.0.0.1:54808 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56492 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56498 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Note the output of this line: `print('Public URL:', ngrok_tunnel.public_url)`. Your ngrok url should be something like `https://fafc-216-228-186-15.ngrok-free.app/`.

Verify that your endpoint is up and running by going to it in your browser: `https://ngrok_url/api/reports/sales?sale_id=1`

Click "Visit Site", then verify that you see output like
```
{
  "id": 1,
  "date": "2023-01-01",
  "amount": 100.0
}
```

In [ ]:
# TODO - figure out a way to get another kernel going for this webbrowser.open(f"({ngrok_url}/api/reports/sales?sale_id=1")

Since we're using FastAPI, we automatically have an OpenAPI spec generated for us at `your-ngrok-endpoint/openapi.json`.
Verify you can see your docs by going to your ngrok `openapi.json` endpoint: `https://ngrok_url/openapi.json`.

In [ ]:
# TODO - webbrowser.open(f"({ngrok_url}/openapi.json")

## 5. Get a Flank API key. Woo!
## WIP
<!-- Navigate to [flank.cloud](flank.cloud) and log in.

Go to **Create Resource** and choose **API**. Choose **Add your own API**. Paste in docs endpoint for the **API Specs URL**, and nickname the API whatever you'd like.

Remember, your docs endpoint should be something like `https://fafc-216-228-186-15.ngrok-free.app/openapi.json`.

Next, sync your endpoint. Syncing just means Flank automatically finds the endpoints in your API. You can pick and choose the ones you'd like to add. -->

## 6. Publish to Flank. Yes!!
Now, you've got a dedicated page webpage on which you can run your endpoint (we call these endpoints "commands" in Flank).

Try running your endpoint!

And just like that, you've got an interface to get your sales report.

Notice that Flank automatically found the parameters (`sale_id`). Flank will find parameters specified in API specs.

If you want to further configure the command (make the name more user-friendly, add a note to it, hide/lock certain parameters), you can do so here.

You can share your command with others and they can run your endpoint without you needing to build an interface for them.

Note that your ngrok tunnel will shut down when you close this notebook. But you can put any piece of deployed code on Flank - a stored procedure, a cloud function, a deployed API.

### What will you Flank?